# Human-based (interpretation) modeling
this notebook is a use case application to model a simplifed geological configuration using a human based interpretation (interpolation-based appraoch)

## Methods


In [ ]:
import numpy as np
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging



def read_csv_and_extract_arrays(csv_file_path, column_names, encoding = 'utf-8' , sep = ';'):
    """
    Read a CSV file into a DataFrame and extract specified columns as NumPy arrays.

    Parameters:
    - csv_file_path (str): Path to the CSV file.
    - column_names (list): List of column names to extract.

    Returns:
    - tuple of NumPy arrays: Tuple containing NumPy arrays for each specified column.

    Raises:
    - ValueError: If a specified column is missing or has an incorrect length.
    """

    # Read CSV file into a DataFrame
    df = pd.read_csv(csv_file_path, encoding= encoding, sep = sep)

    # Extract columns as NumPy arrays
    arrays = tuple(df[column_name].values for column_name in column_names)

    return arrays
def render_points(x, y, z, show_legend=False, save_path = None, **kwargs):
    # Plotting
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot original data points
    ax.scatter(x, y, z, color='black', label='Original Data')

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')
    
    # Set view angles
    ax.view_init(elev=kwargs.get('elev', 0), azim=kwargs.get('azim', -90))

    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    ax.set_xticks(np.arange(kwargs.get('x_t_min', 0), kwargs.get('x_t_max', 220), kwargs.get('x_ar_lag', 100)))
    ax.set_yticks(np.arange(kwargs.get('y_t_min', 0), kwargs.get('y_t_max', 60), kwargs.get('y_ar_lag', 50)))
    ax.set_zticks(np.arange(kwargs.get('z_t_min', -60), kwargs.get('z_t_max', -20), kwargs.get('z_ar_lag', 20)))

    if show_legend:
        ax.legend()

   
    if save_path is not None:
        plt.savefig(os.path.join(save_path, '{}.png'.format(kwargs.get('name_of_fig', 'figure'))), dpi=kwargs.get('dpi', 300), transparent=kwargs.get('transparent', True))
   
    plt.show()

def calculate_rbf(x, y, z, dip, dip_dir, function='linear', smooth=0):
    """
    Calculate interpolation using radial basis functions (RBF) with dip and dip direction as gradients.

    Parameters:
    - x, y, z (arrays): Coordinates of the original 3D point data.
    - dip, dip_dir (arrays): Dip and dip direction values.
    - function (str): Type of RBF function to use for interpolation (default is 'linear').
    - smooth (float): Smoothing parameter for RBF interpolation (default is 0).

    Returns:
    - rbf: Rbf object for interpolation.
    """
    dz_dip = np.sin(dip)
    dz_dip_dir = np.cos(dip) * np.sin(dip_dir)
    dz_dip_dir /= np.cos(dip)  # Normalize by cosine of dip to avoid vertical exaggeration

    # Interpolate using Rbf with dip and dip direction as gradients
    rbf = Rbf(x, y, z + dz_dip + dz_dip_dir, function=function, smooth=smooth)
    return rbf


def render_3d_plot_rbf(x, y, z, rbf, grid_size=500,  
                       cmap='viridis', c='green', alpha=1, show_legend=False, label='Interpolated Surface',
                       save_path = None, **kwargs):
    """
    Render a 3D plot with original data points and the interpolated surface.

    Parameters:
    - x, y, z (arrays): Coordinates of the original 3D point data.
    - rbf: Rbf object for interpolation.
    - grid_size (int): Number of points in the grid for visualization (default is 100).
    - cmap (str): Colormap for the interpolated surface (default is 'viridis').
    - alpha (float): Transparency of the interpolated surface (default is 0.5).
    - **kwargs: Additional keyword arguments for customization.
    """
    # Create a meshgrid for visualization
    x_grid, y_grid = np.meshgrid(np.linspace(x.min(), x.max(), grid_size), np.linspace(y.min(), y.max(), grid_size))
    z_grid = rbf(x_grid.flatten(), y_grid.flatten())

    # Reshape z_grid to match the shape of x_grid and y_grid
    z_grid = z_grid.reshape(x_grid.shape)

    # Plotting
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot original data points
    ax.scatter(x, y, z, color='black', label='Original Data')

    # Plot interpolated surface
    if cmap is not None:
        # Plot the surface
        ax.plot_surface(x_grid, y_grid, z_grid, cmap=cmap, alpha=alpha, label=label)
    else:
        ax.plot_surface(x_grid, y_grid, z_grid, color=c, alpha=alpha, label=label)

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')
    
    # Set view angles
    ax.view_init(elev=kwargs.get('elev', 0), azim=kwargs.get('azim', -90))

    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    ax.set_xticks(np.arange(kwargs.get('x_t_min', 0), kwargs.get('x_t_max', 220), kwargs.get('x_ar_lag', 100)))
    ax.set_yticks(np.arange(kwargs.get('y_t_min', 0), kwargs.get('y_t_max', 60), kwargs.get('y_ar_lag', 50)))
    ax.set_zticks(np.arange(kwargs.get('z_t_min', -60), kwargs.get('z_t_max', -20), kwargs.get('z_ar_lag', 20)))

    if show_legend:
        ax.legend()

    if save_path is not None:
        plt.savefig(os.path.join(save_path, '{}.png'.format(kwargs.get('name_of_fig', 'figure'))), dpi=kwargs.get('dpi', 300), transparent=kwargs.get('transparent', True))
   
    plt.show()


def perform_kriging(x, y, z, variogram_model='gaussian', nlags=10, verbose = True
                   , min_gridx = -10, max_gridx = 220,
                   min_gridy = -10, max_gridy = 60, arr_fact = 3):
    """
    Perform Ordinary Kriging on 3D point data.

    Parameters:
    - x, y, z (arrays): Coordinates of the 3D point data.
    - variogram_model (str): Type of variogram model to use (default is 'gaussian').
    - nlags (int): Number of lags to consider for the variogram model (default is 10).

    Returns:
    - OK: OrdinaryKriging object.
    - gridx, gridy (arrays): Coordinates of the grid for interpolation.
    - zstar, ss: Kriging results and variance.
    """
    OK = OrdinaryKriging(x, y, z, variogram_model=variogram_model, nlags=nlags, verbose = verbose)
    
    gridx = np.arange(min_gridx, max_gridx, arr_fact, dtype='float64')
    gridy = np.arange(min_gridy, max_gridy, arr_fact, dtype='float64')
    
    zstar, ss = OK.execute("grid", gridx, gridy)

    return OK, gridx, gridy, zstar, ss

def render_kriging_3d_plot(x, y, z, OK, gridx, gridy, zstar, cmap='viridis', c='blue', show_legend=False, save_path = None, **kwargs):
    """
    Render a 3D plot of kriging results.

    Parameters:
    - OK: OrdinaryKriging object.
    - gridx, gridy (arrays): Coordinates of the grid for interpolation.
    - zstar: Kriging results.
    - cmap (str): Colormap for the surface plot (default is 'viridis').
    - **kwargs: Additional keyword arguments for customization.
    """
    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create meshgrid for 3D plot
    X, Y = np.meshgrid(gridx, gridy)

    if cmap is not None:
        # Plot the surface
        surf = ax.plot_surface(X, Y, zstar, cmap=cmap)
    else:
        surf = ax.plot_surface(X, Y, zstar, color=c)

    ax.scatter(x, y, z, color='black', label='Original Data')

    # Set labels for axes
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')

    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    ax.set_xticks(np.arange(kwargs.get('x_t_min', 0), kwargs.get('x_t_max', 220), kwargs.get('x_ar_lag', 100)))
    ax.set_yticks(np.arange(kwargs.get('y_t_min', 0), kwargs.get('y_t_max', 60), kwargs.get('y_ar_lag', 50)))
    ax.set_zticks(np.arange(kwargs.get('z_t_min', -60), kwargs.get('z_t_max', -20), kwargs.get('z_ar_lag', 20)))

    # Set viewing angles
    ax.view_init(elev=kwargs.get('elev', 0), azim=kwargs.get('azim', -90))

    if show_legend:
        ax.legend()
        plt.title('3D Kriging Plot')


    if save_path is not None:
        plt.savefig(os.path.join(save_path, '{}.png'.format(kwargs.get('name_of_fig', 'figure'))), dpi=kwargs.get('dpi', 300), transparent=kwargs.get('transparent', True))
   
    # Show the plot
    plt.show()



def render_kriging_map(OK, gridx, gridy, zstar, x, y, cmap='viridis'
                       , show_legend = False, name_of_fig = 'figure', save = False,dpi = 300, transparent = True ):
    """
    Render a kriging map with scatter plot of input data.

    Parameters:
    - OK: OrdinaryKriging object.
    - gridx, gridy (arrays): Coordinates of the grid for interpolation.
    - zstar, ss: Kriging results and variance.
    - x, y (arrays): Coordinates of the input data.
    - cmap (str): Colormap for the interpolation map (default is 'viridis').
    """
    # Plot interpolation map
    cax = plt.imshow(zstar, extent=(gridx.min(), gridx.max(), gridy.min(), gridy.max()), origin='lower', cmap=cmap)

    # Scatter plot of input data
    plt.scatter(x, y, c='k', marker='.')

    # Add colorbar
    cbar = plt.colorbar(cax)
    cbar.set_label('Kriging Value')

    # Set title
    if show_legend :
        ax.legend()
        plt.title('Interpolation Map')
    if save == True : plt.savefig('{}.png'.format(name_of_fig), dpi=dpi, transparent=transparent)
    # Show the plot
    plt.show()


## Tests

### Inputs querrying

In [ ]:
path = 'D:\ownCloud\GeologicalInterpretor_Proof\inputs\sparse_data.csv'
path1 = 'D:\ownCloud\GeologicalInterpretor_Proof\inputs\sparse_data1.csv'
path2 = 'D:\ownCloud\GeologicalInterpretor_Proof\inputs\sparse_data2.csv'

# Example usage:
try:
    x, y, z, dip, dip_dir = read_csv_and_extract_arrays(path, ['x', 'y', 'z', 'dip', 'dip_dir'])
except ValueError as e:
    print(f"Error: {e}")
df = pd.DataFrame({
    'x': x,
    'y': y,
    'z': z,
    'dip': dip,
    'dip_dir': dip_dir,
})


# Example usage:
try:
    x1, y1, z1, dip1, dip_dir1 = read_csv_and_extract_arrays(path1, ['x', 'y', 'z', 'dip', 'dip_dir'])
except ValueError as e:
    print(f"Error: {e}")
df1 = pd.DataFrame({
    'x1': x1,
    'y1': y1,
    'z1': z1,
    'dip1': dip1,
    'dip_dir1': dip_dir1,
})



# Example usage:

try:
    x2, y2, z2, dip2, dip_dir2 = read_csv_and_extract_arrays(path2, ['x', 'y', 'z', 'dip', 'dip_dir'])
except ValueError as e:
    print(f"Error: {e}")
df2 = pd.DataFrame({
    'x2': x2,
    'y2': y2,
    'z2': z2,
    'dip2': dip2,
    'dip_dir2': dip_dir2,
})



### kriging-based

In [ ]:



OK_example, gridx_example, gridy_example, zstar_example, ss_example = perform_kriging(x, y, z, 
                                                min_gridx = -10, max_gridx = 220,
                                                       min_gridy = 0, max_gridy = 40, arr_fact = 3 )
render_kriging_3d_plot(x, y, z,OK_example, gridx_example, gridy_example, zstar_example, cmap = None, 
                       save_path = saving_path, name_of_fig = 'OK_example', **yy_view)


In [ ]:



OK_example1, gridx_example1, gridy_example1, zstar_example1, ss_example1 = perform_kriging(x1, y1, z1)
render_kriging_3d_plot( x1, y1, z1,OK_example1, gridx_example1, gridy_example1, zstar_example1, cmap = None, 
                       save_path = saving_path, name_of_fig = 'OK_example1', **top_view)


In [ ]:

OK_example2, gridx_example2, gridy_example2, zstar_example2, ss_example2 = perform_kriging(x2, y2, z2)
render_kriging_3d_plot(x2, y2, z2,OK_example2, gridx_example2, gridy_example2, zstar_example2, cmap = None, 
                       save_path = saving_path, name_of_fig = 'OK_example2',**inclinde_view)


### views


In [ ]:
  
inclinde_view = {'elev': 35, 'azim':-75}
yy_view = {'elev': 0, 'azim':-90}
top_view = {'elev': 90, 'azim':0}

saving_path = 'D:\ownCloud\GeologicalInterpretor_Proof\human_based_scenario\outputs'
#saving_path = None

view = inclinde_view
suffix = str('inclinde_view')


### RBF-based


render_points(x, y,z, save_path = saving_path, x_ar_lag =25, z_ar_lag = 5,  **view)

In [ ]:
rbf_example = calculate_rbf(x, y, z, dip, dip_dir)
render_3d_plot_rbf(x, y, z, rbf_example, cmap = None, 
                       save_path = saving_path, name_of_fig = 'rbf_example{}'.format(suffix), **view)

In [ ]:
rbf_example1 = calculate_rbf(x1, y1, z1, dip1, dip_dir1)
render_3d_plot_rbf(x1, y1, z1, rbf_example1 ,  cmap = None, 
                       save_path = saving_path, name_of_fig = 'rbf_example1{}'.format(suffix), **view)

In [ ]:
rbf_example2 = calculate_rbf(x2, y2, z2, dip2, dip_dir2)
render_3d_plot_rbf(x2, y2, z2, rbf_example2,  cmap = None, 
                       save_path = saving_path, name_of_fig = 'rbf_example2{}'.format(suffix), **view)